This notebook loads storm data (dates, and ROI geoJSON string) and generates floodmaps from it. It then exports these to Google Drive and Google Earth Engine. Code to replicate the floodmaps of [Canty et al.](https://doi.org/10.3390/rs12010046) using this [tutorial](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-4). Use with the`hybridmodels` conda environment.

## Flood mapping
The flood mapping method utilises information about the distribution of the pixels in the VH and VV to identify significant changes between snapshots. In all VV, VH, HV, and HH bands, water has low backscatter compared to soil and bare vegetation and so amplitude-based methods can be used to detect changes [(Bonafilia 2020)](http://dx.doi.org/10.1109/CVPRW50498.2020.00113). This method is not so good for urban areas, however, and InSAR-based methods for urban areas by [Chini (2019)](https://doi.org/10.3390/rs11020107) will be added later. 

Briefly, pixels in the $m$-look multi-look images have a gamma distribution with shape parameters $m$ and $a/m$, where $a$ is the average intensity. Change between images is tested for using likelihood ratio tests (LRTs), where a small test statistic $Q$ indicates a significant difference between image likelihoods under the two hypotheses of no change/change. The distribution of this test statistic cannot be derived analytically so is approximated using Wilk's Theorem for the $-2\log Q$ transformation, which tends to a chi-squared distribution for many measurements. Further to this, a process for identifying changes over a sequence of images (omnibus test) is derived by factorising $Q$ into intervals $R_j$ whose product is $Q$.

## Manual input:
* Time frame and region geojson from [this link](geojson.io).

In [ ]:
# authenticate Google Earth Engine account
import ee
ee.Authenticate()

# workaround to solve conflict with collections
import collections
collections.Callable = collections.abc.Callable

ee.Initialize(project="floodmapping-2022")

In [127]:
# final import cell
import folium
import numpy as np
from os.path import join
import geopandas as gpd
import floodmap_utils as tut
from functools import partial

In [128]:
# Add EE drawing method to folium.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True):
    """Method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True,
    show = show
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer


In [160]:
import pandas as pd
from shapely.geometry import shape
from ast import literal_eval

event_df = pd.read_csv(join("..", "data", "event_data.csv"))
meta = {}

storm_id = 3
storm, region = event_df['name'][storm_id].split('_')
startdate = event_df['startdate'][storm_id]
enddate = event_df['enddate'][storm_id]

geoJSON = event_df['geojson'][storm_id]
geoJSON = literal_eval(geoJSON)

save_dir = join("..", "data", "floodmaps")
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords,
                          proj=ee.Projection('EPSG:4326'))

polys = [shape(x['geometry']) for x in geoJSON['features']]
aoi_df = gpd.GeoDataFrame({"geometry": polys})
aoi_lonlat = aoi_df.set_crs("EPSG:4326")

print(f"Storm {storm} in {region} from {startdate} to {enddate}.")

Storm roanu in bangladesh04 from 2016-05-17 to 2016-05-22.


In [161]:
# update meta info
aoi_pm = aoi_lonlat.to_crs('EPSG:3857')

height = aoi_pm.total_bounds[3] - aoi_lonlat.to_crs('EPSG:3857').total_bounds[1]
width = aoi_pm.total_bounds[2] - aoi_lonlat.to_crs('EPSG:3857').total_bounds[0]

meta['width (m)'] = width
meta['height (m)'] = height
meta['area (sqm)'] = aoi_pm.area[0]
print(meta)

{'width (m)': 87443.96035823971, 'height (m)': 79494.50941658253, 'area (sqm)': 6951314730.121356}


# Import GEE Collection
Print the orbit numbers and orbit passes of images within the timeframe. Since this is change detection, we need images to have the same orbit passes and numbers. Then filter by the most frequent orbit numbers and passes in order to get most images.

In [162]:
def day_mosaics(date, newlist):
    """https://gis.stackexchange.com/questions/280156/mosaicking-image-collection-by-date-day-in-google-earth-engine"""
    # cast
    datestr = date
    date = ee.Date(date)
    newlist = ee.List(newlist)
    
    # filter collection between date and next day
    filtered = im_coll.filterDate(date, date.advance(1, 'day'))
    
    # make the mosaic
    image = (ee.Image(filtered.mosaic())
             .copyProperties(filtered, filtered.propertyNames()))
    image = image.set({"date": date.format('YYYYMMdd')})

    # Add the mosaic to a list only if the collection has images
    return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image), newlist))

In [163]:
import statistics
startbuffer = ee.Date(startdate).advance(-1, 'month')
endbuffer = ee.Date(enddate).advance(1, 'month')

# define image collection
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
           .filterBounds(aoi)
           .filterDate(startbuffer, endbuffer)
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

# choose the most represented orbit passes
orbit_passes = (im_coll.aggregate_array('orbitProperties_pass').getInfo())
orbit_nums = (im_coll.aggregate_array('relativeOrbitNumber_start').getInfo())

orbit_num = statistics.mode(orbit_nums)
imode = orbit_nums.index(orbit_num)
orbit_pass = orbit_passes[imode]

im_coll = (im_coll.filter(ee.Filter.eq('orbitProperties_pass', orbit_pass))
           .filter(ee.Filter.eq('relativeOrbitNumber_start', orbit_num)))

# supply aoi to clip_img function
clip_img = partial(tut.clip_img, aoi=aoi)

im_list = im_coll.toList(im_coll.size())
im_list = ee.List(im_list.map(clip_img))

timestamplist = (im_coll.aggregate_array('date')
                 .map(lambda d: ee.String('T').cat(ee.String(d)))
                 .getInfo())

# mosaic all/any images taken on the same day
start = startbuffer
end = endbuffer
diff = end.difference(start, 'day')

# all the dates
drange = ee.List.sequence(0, diff.subtract(1)).map(lambda day: start.advance(day, 'day'))

im_coll = ee.ImageCollection(ee.List(drange.iterate(day_mosaics, ee.List([]))))
im_list = im_coll.toList(im_coll.size())
im_list = ee.List(im_list.map(clip_img))

timestamplist = (im_coll.aggregate_array('date')
                 .map(lambda d: ee.String('T').cat(ee.String(d)))
                 .getInfo())

print(timestamplist)

['T20160419', 'T20160513', 'T20160525', 'T20160606']


In [164]:
clip=False

if clip:
    from shapely.geometry import box
    from shapely.affinity import rotate

    def create_box(lat, lon, rotation=0, length=0.1):
        """Create a box around a given lon/lat pair using degrees."""
        # very crude way to manage projections
        b = box(lon-length, lat-length, lon+length, lat+length)
        b = rotate(b, rotation)
        b = ee.Geometry.Polygon(b.__geo_interface__["coordinates"],
                                  proj=ee.Projection("EPSG:4326"))
        return b

    aoi = create_box(*location, rotation=-10)
    clip_img = partial(tut.clip_img, aoi=aoi)
    im_list = ee.List(im_list.map(clip_img))

In [165]:
# create an image of just this band
location = aoi.centroid().coordinates().getInfo()[::-1]
palette = ['black', 'red', 'cyan', 'yellow']

mp = folium.Map(location=location, tiles='Stamen Toner', zoom_start=10)
mp.add_ee_layer(ee.Image(im_list.get(0)).select('VV'), {'min': 0,'max': 3, 'palette': palette}, 'im')

mp.add_child(folium.LayerControl())

## Generate change maps and export to GEE

In [166]:
# Generate the set of change maps as in tutorial

# Run the algorithm with median filter and at 1% significance.
result = ee.Dictionary(tut.change_maps(im_list, median=True, alpha=0.01))

# Extract the change maps and export to assets.
change_maps = ee.Image(result.get('bmap'))
change_maps = change_maps.rename(timestamplist[1:])

The value 2 here corresponds to [negative definite changes](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-3) and correspond to decreases in the intensity of the VV and VH reflectance bands associated with flooding.

## Extract image with most flooding (most pixels=2)

In [167]:
%%time

# Extract the image with the most flooding
# scale=100 takes (wall clock time) 1.79 s, scale=10 takes (wall clock time) 2min 23s
histDict = change_maps.reduceRegion(reducer=ee.Reducer.fixedHistogram(0, 4, 4), geometry=aoi, scale=100, maxPixels=1e9)
histDict = histDict.getInfo()
negdef_dict = {key: value[2][1] for key, value in histDict.items()}
flood_date = max(negdef_dict, key=negdef_dict.get)

print(negdef_dict)
print(f"Biggest flood date: {flood_date}")

{'T20160513': 135, 'T20160525': 13059.235294117647, 'T20160606': 4241.482352941177}
Biggest flood date: T20160525
CPU times: user 16.2 ms, sys: 3.15 ms, total: 19.4 ms
Wall time: 8.87 s


In [168]:
# create an image of just this band, then export it and the date to GEE
location = aoi.centroid().coordinates().getInfo()[::-1]
palette = ['black','red', 'cyan', 'yellow']

im_mean = im_coll.select('VV').mean().clip(aoi)

mp = folium.Map(location=location, tiles='Stamen Toner', zoom_start=11)
mp.add_ee_layer(im_mean, {}, "mean of images")
change_maps = change_maps.updateMask(change_maps.gt(0))
for date in timestamplist[1:]:
    mp.add_ee_layer(change_maps.select(date), {'min': 0,'max': 3, 'palette': palette}, date)

mp.add_child(folium.LayerControl())

In [169]:
# finally, export the maps to Google Earth Engine assets
im_mean = im_coll.select('VV').mean().clip(aoi)

final_maps = change_maps.select(flood_date)

# define path to assets
assetId = f'projects/floodmapping-2022/assets/examples/{region}_{flood_date}'

assexport = ee.batch.Export.image.toAsset(final_maps,
                                      description='assetExportTask',
                                      assetId=assetId, scale=10, maxPixels=1e9)

# export to google earth engine account project folder
assexport.start()


In [176]:
f"{region}_{flood_date}"

'bangladesh04_T20160525'

In [182]:
# check status of export
assexport.status()

{'state': 'READY',
 'description': 'assetExportTask',
 'creation_timestamp_ms': 1652355518148,
 'update_timestamp_ms': 1652355518148,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'EJJ3IYFNH3EAKVYM7LHWOLBA',
 'name': 'projects/floodmapping-2022/operations/EJJ3IYFNH3EAKVYM7LHWOLBA'}

In [172]:
# define path to assets
assexport_drive = ee.batch.Export.image.toDrive(change_maps.select(flood_date),
                                                description='assetExportTask',
                                                fileNamePrefix=f"{storm}_{region}_{flood_date}",
                                                scale=10,
                                                maxPixels=1e9)

# export to google earth engine account
assexport_drive.start()

In [177]:
assexport_drive.status()

{'state': 'READY',
 'description': 'assetExportTask',
 'creation_timestamp_ms': 1652355519822,
 'update_timestamp_ms': 1652355519822,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'V4MAXWYGLDDB2LI7RV7PRS3W',
 'name': 'projects/floodmapping-2022/operations/V4MAXWYGLDDB2LI7RV7PRS3W'}

## Load from GEE and visualise

In [146]:
print(f"{storm}_{region}_{flood_date}")

gee_map = ee.Image(f'projects/floodmapping-2022/assets/examples/{region}_{flood_date}')
gee_map = gee_map.updateMask(gee_map.gt(0))

location = aoi.centroid().coordinates().getInfo()[::-1]
palette = ['black', 'red', 'cyan', 'yellow']

permwater = ee.Image("JRC/GSW1_3/GlobalSurfaceWater").clip(aoi)
mangroves = ee.Image(ee.ImageCollection("LANDSAT/MANGROVE_FORESTS")
                               .filterBounds(b)
                               .first()
                               .clip(aoi))
mangrovesVis = {'min': 0, 'max': 1.0, 'palette': '#2edb7f'}
permwaterVis = {'min': 0.0, 'max': 100, 'palette': ['#ffffff', '#ffbbbb', '#0000ff']}

mp = folium.Map(location=location, tiles='Stamen Toner', zoom_start=11)

mp.add_ee_layer(im_mean, {}, "mean of images")
mp.add_ee_layer(gee_map.select(flood_date), {'min': 0,'max': 3, 'palette': palette}, flood_date)
mp.add_ee_layer(permwater.select('occurrence'), permwaterVis, 'jrc permanent water')
mp.add_ee_layer(mangroves, mangrovesVis, 'mangroves')

mp.add_child(folium.LayerControl())

roanu_bangladesh03_T20160525


In [108]:
# uncomment to save the html file of the flood maps
mp.save(join(save_dir, f"{region}_{flood_date}.html"))